# Pyosys issues

There are a number of pyosys issues that make interactive creation of designs challenging.
In particular, they will cause the IPython kernel to restart without showing an error.

They are in general:

* Spurious exits upon assert()
 * When IdString existing
 * Using an illegal identifier
 * Comparing IdString against None
* Segmentation faults

## Preparation

First, create a simple XOR logic:

In [1]:
from myhdl import *

@block
def test(a, b):
    @always_comb
    def xor_assign():
        b.next = a ^ 0xaa

    return instances()


def convert_verilog(unit):
    a, b = [ Signal(intbv()[8:]) for i in range(2)]
    inst = unit(a, b)
    inst.convert("verilog")
    
convert_verilog(test)

Show Verilog:

In [2]:
!cat test.v

// File: test.v
// Generated by MyHDL 0.11
// Date: Mon May 11 13:39:51 2020


`timescale 1ns/10ps

module test (
    a,
    b
);


input [7:0] a;
output [7:0] b;
wire [7:0] b;





assign b = (a ^ 170);

endmodule


# Function check

In [3]:
from pyosys import libyosys as ys

def addSignal(m, name, n, public = False):
    w = m.addWire(name, n)
    return ys.SigSpec(w)

def ID(x):
    return ys.IdString("$" + x)

d = ys.Design()
m = d.addModule(ID("test1"))


clk, r, d, q = [ addSignal(m, ID("sig_%d" % i), 1) for i in range(4)]

c = ys.Const(0, 1)

m.addAdff(ID("adff"), clk, r, d, q, c, 0, 0)

Cell "$adff"

## Crash scenarios

Before testing the crash scenarios, you'll have to 'prepare' with this auxiliary code snippet (which you will have to rerun once the kernel dies)

In [1]:
from pyosys import libyosys as ys
from io import FileIO


log = FileIO("errlog.txt", "w")

#ys.log_to_stream(log)

def ID(x):
    return ys.IdString("$" + x)

def addSignal(m, name, n, public = False):
    w = m.addWire(name, n)
    return ys.SigSpec(w)

d = ys.Design()
ys.run_pass("read_verilog test.v", d)
ys.run_pass("ls", d)

m = d.addModule(ID("gna"))
s0 = addSignal(m, ID("First"), 1)

## 'assert' exits

### Kernel dies due to assert()

(see jupyter notebook console)

```ERROR: Assert `count_id(wire->name) == 0' failed in kernel/rtlil.cc:1564.```

In [ ]:
s1 = addSignal(m, ID("First"), 1)

### Illegal identifier

In [ ]:
s2 = addSignal(m, ys.IdString("asd"), 4)

# Random segfaulting

### Comparison against None

In [ ]:
a = ID("gna")
if a == None:
    print("No good")